# Principle
## Basics
- objective: $e = s(m(t),a) - s*$, m(t) is the specific image info, a could represent the camera intrinsic like the system parameter, s encodes the image pixel features
- $\dot{e} = \dot{s}$, $\dot{s} = L_sv_c, L_s=L_e$, L_s is the jacobian matrix which describes the relation between camera location and image pixel feature, v_c is the camera velocity including linear and angular
- control law: $v_c = -\lambda L_c^\dagger e$, assuming $\dot{e} = -\lambda e$

## Image jacobian
- Using intrinsic parameters, the pixel coordinate could be remapped back to homogenous camera coordinate $\begin{bmatrix} x  \\
                            y \end{bmatrix} = \begin{bmatrix} \frac{X_c}{Z_c}  \\
                            \frac{Y_c}{Z_c} \end{bmatrix} = \begin{bmatrix} \frac{u-c_u}{f}  \\
                                                \frac{v-c_v}{f} \end{bmatrix}$  
- $L_c =  \begin{bmatrix} \frac{1}{Z} & 0 & -\frac{x}{Z} & -xy & (1+x^2) & -y  \\
                           0 & \frac{1}{Z} & -\frac{y}{Z} & -(1+y^2) & xy & x \end{bmatrix}$, please note one feature point in image could provide two function, so we need at lease three feature points to solve all the componets for camera velocity
                           

In [2]:
import numpy as np
def pixel2homoCam(u, v , fx, cx, fy, cy):
    x = (u - cx) / fx
    y = (v - cy) / fy
    return x, y

# cam_pts should be homogenous camera coordinate feature points
def calImgJac(cam_pts, Z = 1):
    pts = cam_pts.reshae(-1,2)
    num_pts = len(pts)
    jacobian = np.zeros(num_pts*2, 6)
    for i in range(num_pts):
        x = pts[i, 0]
        y = pts[i, 1]
        jacobian[i*2:(i+1)*2] = np.array([[1/Z, 0, -x/Z, -x*y, 1+x*x, -y],
                                          [0, 1/Z, -y/Z, -(1+y*y) ,x*y, x]])
    return jacobian

def ibvs(pixel_pts, ref_pts, camera_param, scaling = 0.001):
    feature_pts = pixel_pts.reshape(-1,2)
    ref_pts = ref_pts.reshape(-1,2)
    assert len(ref_pts) == len(feature_pts), 'Not detected the corresponding reference point'
    error = ref_pts - feature_pts
    error = error.reshape(-1,1) # error becomes 2nx2
    assert len(feature_pts) > 3, 'Could not solve the control law due to lack of feature points'
    for i in range(len(feature_pts)):
        feature_pts[i,0], feature_pts[i,1] = pixel2homoCam(feature_pts[i,0], feature_pts[i,1], 
                                                           camera_param['fx'], camera_param['cx'],
                                                           camera_param['fy'], camera_param['cy'])
        jacobian = calImgJac(feature_pts)
    cam_vel = - scaling * np.dot(np.linalg.pinv(jacobian), error)
    return cam_vel
        